In [1]:
import json
import csv
from tqdm import tqdm_notebook as tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re
import time

In [2]:
words = []
dataset = []
with open("../datasets/dataset.csv", "r") as f:
    tags = csv.reader(f)
    header = next(tags)
    
    for t in tags:
        words += t
        dataset += [t]

In [23]:
# Qiitaからタグ検索
def tagSearch(w, pagesize="15"):
    options = Options()
    options.add_argument('--headless')
    driver = webdriver.Chrome(chrome_options=options)
    driver.get('https://stackoverflow.com/questions/tagged/'+w+'?tab=newest&pagesize='+pagesize)
    data = driver.page_source
    html = BeautifulSoup(data, 'lxml')
    driver.close()
    driver.quit()
    return html

In [29]:
html = tagSearch("ecmascript", pagesize="50")

In [35]:
html.find(class_="s-input")['value']

'[javascript]'

In [42]:
res_d = dict()

for w in tqdm(set(words)):
    res = []
    html = tagSearch(w, pagesize="50")
    tag_list = html.find_all(class_="tags")
    flag = html.find(class_="s-input")['value']
    
    if tag_list and flag[1:-1]==w:
        for i in tag_list:
            res += [[j.text.lower() for j in i.find_all("a")]]
            time.sleep(1.0)

    res_d[w] = res

In [43]:
res_d.keys()

dict_keys(['laravel', 'ecmascript', 'npm', 'redux', 'rstudio', 'phina.js', 'typescript', 'chrome-extension', 'pip', 'vuetify', 'd3.js', 'flask', 'html', 'scipy', 'scikit-learn', 'mruby', 'eslint', 'electron', 'nuxt.js', 'kivy', 'ruby', 'rspec', 'java', 'sass', 'reactnative', 'backbone.js', 'yarn', 'css', 'cakephp', 'beautifulsoup', 'react', 'promise', 'jquery', 'canvas', 'keras', 'ajax', 'webgl', 'coffeescript', 'rails', 'jupyter', 'pyqt', 'numpy', 'rbenv', 'php', 'three.js', 'node.js', 'r', 'javascript', 'vue.js', 'django', 'capybara', 'bootstrap', 'symfony', 'tkinter', 'next.js', 'angular', 'vuex', 'scss', 'webpack', 'javafx', 'pytorch', 'python', 'chainer', 'pyenv', 'spring', 'babel'])

In [44]:
with open('stack_overflow_results_data.json', 'w') as f:
    json.dump(res_d, f)

In [45]:
with open('stack_overflow_results_data.json', 'r') as f:
    read_res_d = json.load(f)

In [48]:
read_res_d["react"]

[]

In [67]:
c = 0
cl = []
for r in read_res_d:
    #print(len(read_res_d[r]))
    if not read_res_d[r]:
        c += 1
        cl.append(r)
        
print(c)
print(cl)

9
['ecmascript', 'phina.js', 'chrome-extension', 'vuetify', 'scss', 'react', 'rails', 'reactnative', 'bootstrap']


In [50]:
# バージョンを表すテキストを削除
def eraseVersion(w):
    w = re.sub('(\d+(?:\.\d+)?)$', '', w)
    return w

In [51]:
# w2に対するw1のスコアを返す
def check(w1, w2):
    w1 = w1.lower()
    w2 = w2.lower()
    res = []
    score = 0
    count = 0
        
    # w1で検索した際にw2が含まれていたら1 それ以外は0
    for i in read_res_d[w1]:
        res += [[eraseVersion(j.lower())==w2 for j in i].count(True)]
        count += 1
    time.sleep(0.1)

    # 検索結果が存在しない場合スコア0を返す
    if len(res) == 0:
        return 0

    # w1で検索した時にw2が含まれる割合
    score += ((len(res)-res.count(0))+1e-7)/(len(res)+1e-7)

    # 1つのページの出現率が30%以下ならスコア0を返す
    #if score/p < 0.2:
    #    return 0
    
    #print(len(res)-res.count(0))
        
    return round(score, 3)

In [63]:
# 2つの単語を比較
def comparisonCheck(w1, w2, boundary=0.2, p=True):
    w1_score = check(w1, w2)
    w1, w2 = w2, w1
    w2_score = check(w1, w2)
    w1, w2 = w2, w1
    
    if p:
        print(w1, "Dscore:", w1_score)
        print(w2, "Dscore:", w2_score)
        
    if (w1_score < boundary and w2_score < boundary) or (abs(w1_score-w2_score) < 0.1):
        if p:
            print(w1, "and", w2, "are irrelevant")
        return {}
    else:
        if w1_score < w2_score:
            if p:
                print(">>", w1, "is easy!")
            return {"easy": w1, "hard": w2, "score": max(w1_score, w2_score)}
        else:
            if p:
                print(">>", w2, "is easy!")
            return {"easy": w2, "hard": w1, "score": max(w1_score, w2_score)}

In [64]:
comparisonCheck("javascript", "vue.js")

javascript Dscore: 0.04
vue.js Dscore: 0.28
>> javascript is easy!


{'easy': 'javascript', 'hard': 'vue.js', 'score': 0.28}

In [66]:
correct = 0
check_res = []

for ds in tqdm(dataset):
    
    cr = comparisonCheck(ds[0], ds[1], p=False)
    if cr:
        if [cr["easy"], cr["hard"]] == [ds[0], ds[1]]:
            correct += 1
        else:
            print([ds[0], ds[1]])
    else:
        print([ds[0], ds[1]])
            
print(correct/(len(dataset)))

['javascript', 'react']
['javascript', 'ecmascript']
['javascript', 'chrome-extension']
['javascript', 'phina.js']
['javascript', 'npm']
['javascript', 'yarn']
['html', 'electron']
['css', 'bootstrap']
['css', 'scss']
['vue.js', 'vuetify']
['webgl', 'three.js']
['react', 'redux']
['react', 'next.js']
['react', 'reactnative']
['ruby', 'rails']
0.75


In [68]:
correct = 0
count = 0
check_res = []

for ds in tqdm(dataset):
    
    if not (ds[0] in cl or ds[1] in cl):
        cr = comparisonCheck(ds[0], ds[1], p=False)
        if cr:
            if [cr["easy"], cr["hard"]] == [ds[0], ds[1]]:
                correct += 1
            else:
                print([ds[0], ds[1]])
        else:
            print([ds[0], ds[1]])
        count += 1

print(correct/count)

['javascript', 'npm']
['javascript', 'yarn']
['html', 'electron']
['webgl', 'three.js']
0.9183673469387755


In [70]:
correct, count

(45, 49)